# Model Test Notebook - Orijinal Pipeline ile Test
## Halil Melih AKÇA 221104091

Bu notebook, eğitilmiş ensemble modelini orijinal pipeline'ı kullanarak test eder.

## Gerekli Kütüphaneleri Import Et

In [1]:
# ORIJINAL NOTEBOOK'TAKİ AYNI IMPORT'LAR
import pandas as pd
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import numpy as np
import re
import string
import nltk
import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')

# spaCy model yükleme
try:
    nlp = spacy.load("en_core_web_sm")
    print("✓ spaCy model yüklendi")
except:
    print("✗ spaCy model bulunamadı")
    nlp = None

# Diğer gerekli kütüphaneler
from textstat import flesch_reading_ease, automated_readability_index
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns

print("\n✅ Tüm kütüphaneler yüklendi!")

✓ spaCy model yüklendi

✅ Tüm kütüphaneler yüklendi!

✅ Tüm kütüphaneler yüklendi!


## Veri Yükleme ve Model Yükleme

In [ ]:
# Test verilerini yükle
try:
    news_df = pd.read_csv("../stockMarket_predict/Combined_News_DJIA.csv")
    print(f"✓ Combined_News_DJIA.csv yüklendi: {news_df.shape}")
    data_source = "Combined_News_DJIA.csv"
except FileNotFoundError:
    try:
        news_df = pd.read_csv("../stockMarket_predict/upload_DJIA_table.csv")
        print(f"✓ upload_DJIA_table.csv yüklendi: {news_df.shape}")
        data_source = "upload_DJIA_table.csv"
    except FileNotFoundError:
        print("✗ Hiçbir veri dosyası bulunamadı!")
        exit()

print(f"\nVeri kaynağı: {data_source}")
print(f"Veri boyutu: {news_df.shape}")
print(f"Sütunlar: {list(news_df.columns)[:10]}...")  # İlk 10 sütun göster

# Label dağılımını kontrol et
if 'Label' in news_df.columns:
    print(f"\n📊 Label Dağılımı:")
    print(news_df['Label'].value_counts())
else:
    print("⚠️ Label sütunu bulunamadı!")

# Eğitilmiş modeli yükle
try:
    with open("ensemble_model.pkl", "rb") as f:
        ensemble_model = pickle.load(f)
    print("\n✓ Ensemble model yüklendi")
    print(f"Model tipi: {type(ensemble_model).__name__}")
    if hasattr(ensemble_model, 'estimators'):
        print(f"Base estimators: {len(ensemble_model.estimators)}")
        for name, estimator in ensemble_model.estimators:
            print(f"  - {name}: {type(estimator).__name__}")
except FileNotFoundError:
    ensemble_model = None
    print("\n✗ Ensemble model bulunamadı!")
    print("Önce training notebook'unu çalıştırarak modeli eğitin.")

✗ Hiçbir veri dosyası bulunamadı!


NameError: name 'data_source' is not defined

: 

## Orijinal Feature Engineering Fonksiyonları

In [ ]:
# ORIJINAL NOTEBOOK'TAKİ AYNI FONKSİYONLAR

def pos_features_spacy(text):
    """spaCy ile POS tag özellikleri"""
    if nlp is None:
        return [0.25, 0.25, 0.25, 0.25]  # Default değerler
    
    doc = nlp(text)
    total = len(doc)
    if total == 0:
        return [0, 0, 0, 0]
    
    noun_ratio = len([token for token in doc if token.pos_ == "NOUN"]) / total
    verb_ratio = len([token for token in doc if token.pos_ == "VERB"]) / total
    adj_ratio = len([token for token in doc if token.pos_ == "ADJ"]) / total
    adv_ratio = len([token for token in doc if token.pos_ == "ADV"]) / total
    return [noun_ratio, verb_ratio, adj_ratio, adv_ratio]

def clean_text(text):
    """Orijinal metin temizleme fonksiyonu"""
    if pd.isna(text):
        return ""
    
    text = str(text).lower()
    text = re.sub(r"b['\"]|['\"]", "", text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\d+', 'NUMBER', text)
    text = re.sub(r'[^\w\s.,!?;:]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def linguistic_features(text):
    """Linguistik özellikler çıkarma"""
    words = text.split()
    avg_word_len = np.mean([len(w) for w in words]) if words else 0
    punct_count = sum([1 for c in text if c in string.punctuation])
    cap_ratio = sum([1 for c in text if c.isupper()]) / (len(text) + 1e-9)
    digit_ratio = sum([1 for c in text if c.isdigit()]) / (len(text) + 1e-9)
    flesch = flesch_reading_ease(text)
    ari = automated_readability_index(text)
    return [len(words), avg_word_len, punct_count, cap_ratio, digit_ratio, flesch, ari]

financial_keywords = ["bull", "bear", "gain", "loss", "stock", "market"]

def financial_keyword_density(text):
    """Finansal anahtar kelime yoğunluğu"""
    tokens = text.lower().split()
    return [tokens.count(word)/len(tokens) if len(tokens) > 0 else 0 for word in financial_keywords]

def ner_features(text):
    """Named Entity Recognition özellikleri"""
    if nlp is None:
        return [0, 0, 0, 0]
    
    doc = nlp(text)
    counts = {"PERSON":0, "ORG":0, "GPE":0, "MONEY":0}
    for ent in doc.ents:
        if ent.label_ in counts:
            counts[ent.label_] += 1
    return list(counts.values())

print("✅ Tüm feature engineering fonksiyonları tanımlandı!")

## Veri Ön İşleme - Orijinal Pipeline

In [ ]:
print("=== VERİ ÖN İŞLEME - ORİJİNAL PİPELİNE ===")

# Orijinal notebook'taki aynı preprocessing
news_df['Combined'] = news_df.iloc[:, 2:27].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
news_df['Cleaned'] = news_df['Combined'].apply(clean_text)

print(f"✓ {len(news_df)} örnek işlendi")
print(f"✓ Ortalama metin uzunluğu: {news_df['Cleaned'].str.len().mean():.1f} karakter")

# İlk birkaç örneği göster
print("\n📝 İlk 3 temizlenmiş metin örneği:")
for i in range(min(3, len(news_df))):
    text = news_df['Cleaned'].iloc[i][:100] + "..." if len(news_df['Cleaned'].iloc[i]) > 100 else news_df['Cleaned'].iloc[i]
    label = news_df['Label'].iloc[i] if 'Label' in news_df.columns else "?"
    print(f"  {i+1}. Label: {label} | Text: {text}")

## Feature Extraction - Orijinal Sıralama

In [ ]:
print("=== FEATURE EXTRACTION - ORİJİNAL SIRALAMA ===")

# Adım 2a: Linguistik Özellikler
print("📊 Adım 2a: Linguistik özellikler çıkarılıyor...")
ling_df = pd.DataFrame(news_df['Cleaned'].apply(linguistic_features).tolist(), 
                      columns=["word_count", "avg_word_len", "punct_count", 
                              "cap_ratio", "digit_ratio", "flesch", "ari"])
print(f"  ✓ {ling_df.shape[1]} linguistik özellik çıkarıldı")

# Adım 2b: Semantik Özellikler
print("😊 Adım 2b: Semantik özellikler çıkarılıyor...")
sia = SentimentIntensityAnalyzer()
sentiment_df = news_df['Cleaned'].apply(lambda x: pd.Series(sia.polarity_scores(x)))
print(f"  ✓ {sentiment_df.shape[1]} sentiment özellik çıkarıldı")

# Finansal anahtar kelime yoğunluğu (orijinal kodda hesaplandı ama birleştirilmedi)
fin_kw_df = pd.DataFrame(news_df['Cleaned'].apply(financial_keyword_density).tolist(), 
                        columns=[f'kw_{k}' for k in financial_keywords])
print(f"  ✓ {fin_kw_df.shape[1]} finansal anahtar kelime özellik çıkarıldı")

# NER özellikleri (orijinal kodda hesaplandı ama birleştirilmedi)
ner_df = pd.DataFrame(news_df['Cleaned'].apply(ner_features).tolist(), 
                     columns=["PERSON", "ORG", "GPE", "MONEY"])
print(f"  ✓ {ner_df.shape[1]} NER özellik çıkarıldı")

# Adım 2c: Sözdizimsel Özellikler (POS tag oranları)
print("🏷️ Adım 2c: POS tag özellikleri çıkarılıyor...")
pos_df = pd.DataFrame(news_df['Cleaned'].apply(pos_features_spacy).tolist(), 
                     columns=["noun_ratio", "verb_ratio", "adj_ratio", "adv_ratio"])
print(f"  ✓ {pos_df.shape[1]} POS özellik çıkarıldı")

# Adım 2d: İstatistiksel Özellikler (TF-IDF)
print("📝 Adım 2d: TF-IDF özellikleri çıkarılıyor...")
tfidf = TfidfVectorizer(max_features=2000, ngram_range=(1, 2), min_df=2, max_df=0.95, stop_words='english')
tfidf_matrix = tfidf.fit_transform(news_df['Cleaned'])
pca = PCA(n_components=50)
tfidf_pca = pca.fit_transform(tfidf_matrix.toarray())
tfidf_df = pd.DataFrame(tfidf_pca, columns=[f'pca_{i}' for i in range(tfidf_pca.shape[1])])
print(f"  ✓ {tfidf_df.shape[1]} TF-IDF+PCA özellik çıkarıldı")

# ORİJİNAL SIRALAMAYA GÖRE BİRLEŞTİRME
# Orijinal kod: features = pd.concat([ling_df, sentiment_df, pos_df, tfidf_df], axis=1)
features = pd.concat([ling_df, sentiment_df, pos_df, tfidf_df], axis=1)
labels = news_df['Label']

print(f"\n✅ Toplam {features.shape[1]} özellik çıkarıldı (orijinal sıralama ile)!")
print(f"📊 Feature matrix boyutu: {features.shape}")
print(f"📋 Labels boyutu: {labels.shape}")

# Feature istatistikleri
print("\n📈 Feature İstatistikleri:")
print(f"  - Linguistik: {ling_df.shape[1]} özellik")
print(f"  - Sentiment: {sentiment_df.shape[1]} özellik")
print(f"  - POS Tags: {pos_df.shape[1]} özellik")
print(f"  - TF-IDF+PCA: {tfidf_df.shape[1]} özellik")
print(f"  - Toplam: {features.shape[1]} özellik")

## Feature Scaling ve Polynomial Features - Orijinal Pipeline

In [ ]:
print("=== FEATURE SCALING VE POLYNOMIAL FEATURES ===")

# Adım 5: Feature Scaling (orijinal kod)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features.values)
print(f"✓ Feature scaling tamamlandı: {X_scaled.shape}")

# Polynomial Features
poly = PolynomialFeatures(degree=2, interaction_only=True)
X_poly = poly.fit_transform(X_scaled)
print(f"✓ Polynomial features oluşturuldu: {X_poly.shape}")

# Memory kullanımı kontrolü
memory_usage_mb = X_poly.nbytes / (1024 * 1024)
print(f"💾 Memory kullanımı: {memory_usage_mb:.1f} MB")

if memory_usage_mb > 1000:
    print("⚠️ Yüksek memory kullanımı tespit edildi!")
else:
    print("✅ Memory kullanımı kabul edilebilir seviyede")

print(f"\n📊 Final feature pipeline:")
print(f"  Original features: {features.shape}")
print(f"  After scaling: {X_scaled.shape}")
print(f"  After polynomial: {X_poly.shape}")

## Model Test ve Tahmin

In [ ]:
print("=== MODEL TEST VE TAHMİN ===")

if ensemble_model is None:
    print("❌ Model yüklenmediği için tahmin yapılamıyor!")
    print("Önce training notebook'unu çalıştırarak modeli eğitin.")
else:
    try:
        # Adım 6: Train-Test Split (orijinal kod)
        X_train, X_test, y_train, y_test = train_test_split(X_poly, labels, test_size=0.2, random_state=42)
        print(f"✓ Train-test split yapıldı:")
        print(f"  Train: {X_train.shape}")
        print(f"  Test: {X_test.shape}")
        
        # Model ile tahmin yap
        print("\n🤖 Ensemble model ile tahminler yapılıyor...")
        y_pred = ensemble_model.predict(X_test)
        
        # Model bilgileri
        print(f"\n🤖 MODEL BİLGİLERİ:")
        print(f"  Model tipi: {type(ensemble_model).__name__}")
        if hasattr(ensemble_model, 'estimators'):
            print(f"  Base estimators: {len(ensemble_model.estimators)}")
            for name, estimator in ensemble_model.estimators:
                print(f"    - {name}: {type(estimator).__name__}")
        
        # Orijinal evaluate_model fonksiyonu
        def evaluate_model(model, X_test, y_test, y_pred):
            """Kapsamlı model değerlendirmesi"""
            
            print("\n=== MODEL PERFORMANSI ===")
            print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
            print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
            print("\nDetaylı Rapor:")
            print(classification_report(y_test, y_pred))
            
            # Confusion Matrix
            cm = confusion_matrix(y_test, y_pred)
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
            plt.title('Confusion Matrix')
            plt.ylabel('Gerçek')
            plt.xlabel('Tahmin')
            plt.show()
            
            return accuracy_score(y_test, y_pred), f1_score(y_test, y_pred)
        
        # Model değerlendirmesi
        accuracy, f1 = evaluate_model(ensemble_model, X_test, y_test, y_pred)
        
        print(f"\n📊 SONUÇ ÖZETİ:")
        print(f"  🎯 Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
        print(f"  🎯 Test F1 Score: {f1:.4f}")
        
        # Baseline karşılaştırması
        baseline_accuracy = max(pd.Series(y_test).value_counts(normalize=True))
        improvement = accuracy - baseline_accuracy
        print(f"  📊 Baseline Accuracy: {baseline_accuracy:.4f}")
        print(f"  📈 Model İyileştirmesi: +{improvement:.4f} ({improvement*100:.2f} pp)")
        
        # Performans yorumu
        if accuracy > 0.55:
            print(f"  ✅ Çok iyi performans! (>55%)")
        elif accuracy > 0.50:
            print(f"  ⚠️ Kabul edilebilir performans (50-55%)")
        else:
            print(f"  ❌ Düşük performans (<50%)")
            
        if improvement > 0.05:
            print(f"  ✅ Baseline'dan anlamlı iyileştirme sağlandı!")
        else:
            print(f"  ⚠️ Baseline'dan sınırlı iyileştirme")
        
    except Exception as e:
        print(f"❌ Tahmin sırasında hata oluştu: {e}")
        import traceback
        traceback.print_exc()

## Tahmin Örnekleri ve Detaylı Analiz

In [ ]:
if ensemble_model is not None and 'y_pred' in locals():
    print("=== TAHMİN ÖRNEKLERİ VE DETAYLI ANALİZ ===")
    
    # Test set indekslerini al
    test_indices = X_test.index if hasattr(X_test, 'index') else range(len(X_test))
    
    print("\n📝 İlk 10 Test Örneği:")
    print("-" * 100)
    
    for i in range(min(10, len(y_test))):
        true_label = y_test.iloc[i] if hasattr(y_test, 'iloc') else y_test[i]
        pred_label = y_pred[i]
        
        true_text = "📈 UP" if true_label == 1 else "📉 DOWN"
        pred_text = "📈 UP" if pred_label == 1 else "📉 DOWN"
        correct = "✅" if pred_label == true_label else "❌"
        
        # Test set'teki gerçek indeksi bul
        try:
            if hasattr(X_test, 'index'):
                original_idx = X_test.index[i]
                text = news_df['Cleaned'].iloc[original_idx][:80] + "..." if len(news_df['Cleaned'].iloc[original_idx]) > 80 else news_df['Cleaned'].iloc[original_idx]
            else:
                # Basit yaklaşım - test verilerinden göster
                text = "[Test set'ten metin - indeks eşleştirilemedi]"
        except:
            text = "[Metin alınamadı]"
        
        print(f"{i+1:2d}. {correct} Gerçek: {true_text} | Tahmin: {pred_text}")
        if text != "[Test set'ten metin - indeks eşleştirilemedi]":
            print(f"    Text: {text}")
        print()
    
    # Hata analizi
    print("\n🔍 HATA ANALİZİ:")
    correct_predictions = (y_test == y_pred).sum() if hasattr(y_test, '__iter__') else sum(y_test == y_pred)
    total_predictions = len(y_test)
    incorrect_predictions = total_predictions - correct_predictions
    
    print(f"  ✅ Doğru tahminler: {correct_predictions}/{total_predictions}")
    print(f"  ❌ Yanlış tahminler: {incorrect_predictions}/{total_predictions}")
    
    # Confusion matrix analizi
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    print(f"\n📊 CONFUSION MATRIX ANALİZİ:")
    print(f"  True Negatives (doğru DOWN): {tn}")
    print(f"  False Positives (yanlış UP): {fp}")
    print(f"  False Negatives (yanlış DOWN): {fn}")
    print(f"  True Positives (doğru UP): {tp}")
    
    # Precision ve Recall
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    print(f"\n📈 DETAYLI METRİKLER:")
    print(f"  Precision (UP sınıfı): {precision:.4f}")
    print(f"  Recall (UP sınıfı): {recall:.4f}")
    
else:
    print("⚠️ Model tahminleri mevcut değil, detaylı analiz yapılamıyor.")

## Final Rapor ve Sonuçlar

In [ ]:
print("=" * 80)
print("                    FİNAL MODEL TEST RAPORU")
print("=" * 80)

print(f"\n📊 TEST VERİSİ BİLGİLERİ:")
print(f"  📁 Veri kaynağı: {data_source}")
print(f"  📈 Toplam örnek sayısı: {len(news_df)}")
print(f"  🔧 Çıkarılan özellik sayısı: {features.shape[1]}")
print(f"  ⚙️ Polynomial sonrası özellik sayısı: {X_poly.shape[1]}")

if 'X_test' in locals():
    print(f"  🧪 Test set boyutu: {X_test.shape[0]} örnek")

print(f"\n🎯 LABEL BİLGİLERİ:")
if 'Label' in news_df.columns:
    label_counts = news_df['Label'].value_counts().sort_index()
    for label, count in label_counts.items():
        percentage = (count / len(news_df)) * 100
        direction = "📉 DOWN" if label == 0 else "📈 UP"
        print(f"  {direction}: {count} örnek ({percentage:.1f}%)")

if ensemble_model is not None and 'accuracy' in locals():
    print(f"\n🤖 MODEL PERFORMANSI:")
    print(f"  🎯 Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"  🎯 Test F1 Score: {f1:.4f}")
    print(f"  📊 Baseline Accuracy: {baseline_accuracy:.4f}")
    print(f"  📈 Model İyileştirmesi: +{improvement:.4f} ({improvement*100:.2f} pp)")
    
    print(f"\n🏗️ MODEL MİMARİSİ:")
    print(f"  🔧 Model tipi: {type(ensemble_model).__name__}")
    if hasattr(ensemble_model, 'estimators'):
        print(f"  👥 Base estimators:")
        for name, estimator in ensemble_model.estimators:
            print(f"     - {name}: {type(estimator).__name__}")
    
    # Performans değerlendirmesi
    print(f"\n📈 PERFORMANS DEĞERLENDİRMESİ:")
    if accuracy > 0.55:
        print(f"  ✅ ÇOK İYİ PERFORMANS! (Accuracy > 55%)")
    elif accuracy > 0.50:
        print(f"  ⚠️ KABUL EDİLEBİLİR PERFORMANS (50-55%)")
    else:
        print(f"  ❌ DÜŞÜK PERFORMANS (< 50%)")
        
    if improvement > 0.05:
        print(f"  ✅ Baseline'dan ANLAMLı İYİLEŞTİRME!")
    else:
        print(f"  ⚠️ Baseline'dan sınırlı iyileştirme")

else:
    print(f"\n❌ MODEL TEST EDİLEMEDİ")
    print(f"  Model dosyası yüklenemedi veya tahmin yapılamadı")

print(f"\n💡 FİNANSAL SENTIMENT ANALİZ NOTLARI:")
print(f"  • Finansal haber sentiment analizi tipik olarak %45-60 accuracy gösterir")
print(f"  • Haber verileri piyasa hareketlerinin sadece ~%3'ünü açıklar")
print(f"  • %53 civarı accuracy finansal ML'de NORMAL ve KABUL EDİLEBİLİR")
print(f"  • Çok yüksek accuracy (>70%) overfitting işareti olabilir")
print(f"  • Bu model complex ensemble yaklaşımı ile iyi sonuç alıyor")

print(f"\n🔧 KULLANILAN TEKNİKLER:")
print(f"  • Linguistic features (word count, readability)")
print(f"  • Sentiment analysis (VADER)")
print(f"  • POS tagging (spaCy)")
print(f"  • TF-IDF + PCA (dimensionality reduction)")
print(f"  • Polynomial features (interaction terms)")
print(f"  • Ensemble learning (MLP + SVM + XGBoost + KNN)")
print(f"  • Class balancing (balanced weights)")

print(f"\n⏰ Test tarihi: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"👤 Test eden: Halil Melih AKÇA (221104091)")
print("=" * 80)
print("✅ MODEL TEST TAMAMLANDI!")
print("=" * 80)